# Simulation Project. Thermoelectric System.

In [ ]:
# Necessary imports
from App.modules.weibull import Weibull
from App.modules.lognormal import LogNormal
from App.modules.event import Event
from App.modules.thermoelectric import ThermoElectric
from App.modules.agents import Agent
from App.modules.circuit import Circuit
from App.modules.roundrobin import RoundRobin
import random as rnd
import numpy as np
import copy
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import display, Markdown

EPSILON = 1e-8

#### Generating thermoelectrics
##### Weibull Params:
- Shape: random(2, 2.6) Under the assumption that the probability of failure is higher as the system gets older.
- Scale: random(60, 80) Under the assumption that the system will fail at some point between the second and third month of operation.
##### Lognormal Params:
- Mean: random(3.5, 4).
- Variance: random(0.3, 4).
##### Generation Capacity:
- random(200, 1000) megawatts.

In [ ]:
def generate_thermoelectrics(days, thermoelectrics_amount):
    thermoelectrics = []
    for i in range(thermoelectrics_amount):
        w = Weibull(rnd.uniform(60, 80), rnd.uniform(2, 2.6))
        l = LogNormal(rnd.uniform(3.5, 4), rnd.uniform(0.3, 0.4))
        o = rnd.randrange(200, 1000)
        t = ThermoElectric(o, w, l)
        t.planificate_events(days)
        thermoelectrics.append(t)

    return thermoelectrics

Initiallizing some global parameters and the thermoelectric system. The system will be composed of 20 thermoelectrics. Each thermoelectric will have a Weibull and a Lognormal distribution for its lifetime and repair time, respectively. We will be simulating the system for 365 days. The system will hace a storage capacity that will be 0 at the beginning.

In [ ]:
days = 365
thermoelectrics_amount = 20
stored_energy = 0
thermoelectrics = generate_thermoelectrics(days, thermoelectrics_amount)

#### Generating circuits
##### Lognormal Params:
- Mean: random(3, 5).
- Variance: random(0.2, 0.4).

The system will have 115 circuits. Each circuit will have a Lognormal distribution for its dayly demand.

In [ ]:
def generate_circuits(circuits_amount, days):
    circuits = []
    for i in range(circuits_amount):
        demand = LogNormal(rnd.uniform(3, 5), rnd.uniform(0.2, 0.4))
        c = Circuit(i, demand, days)
        circuits.append(c)

    return circuits


# Initialize circuits
circuits_amount = 115
circuits = generate_circuits(circuits_amount, days)

In [ ]:
for i in range(1):
    print(i)
    thermoelectrics[i].plot(0, 365)
    thermoelectrics[i].get_distributions_info()

In [ ]:
# A function for getting the next general event among all thermoelectrics events. An event can be a failure, a maintenance or a repair

def get_next_general_event(thermoelectrics_list, days, current_day):
    next_event = days
    next_thermoelectric = None
    for t in thermoelectrics_list:
        tmp = t.get_next_future_event_day()
        if tmp < next_event and tmp >= current_day:
            next_event = tmp
            next_thermoelectric = t
    return (next_event, next_thermoelectric)

## Simulating

The `simulate` function simulates the operation of a set of thermoelectric devices over a specified number of days.

It takes the following parameters:

- thermoelectrics: a list of thermoelectric devices to simulate.
- days: the number of days to run the simulation.
- agent: an optional agent that can manage the thermoelectrics and circuits.
- circuits: an optional list of circuits that the thermoelectrics are connected to.
- stored_energy: the initial amount of stored energy.
- rotation: the rotation strategy for the circuits.

The function initializes lists to keep track of the working state of the thermoelectrics, the energy deficit per day, and the stored energy per day.

It then enters a loop that runs for the specified number of days. For each day, it calculates the total demand from the circuits, allows the agent to manage the thermoelectrics and circuits, and processes any events that occur on that day.

The function then calculates the total energy offer from the working thermoelectrics and the stored energy, and calculates the energy deficit and stored energy for the day.

Finally, it returns lists representing the working state of the thermoelectrics, the energy deficit per day, the stored energy per day, and the circuits.

In [ ]:
def simulate(
    thermoelectrics: "list[ThermoElectric]",
    days,
    agent: Agent = None,
    circuits: "list[Circuit]" = None,
    stored_energy = 0,
    rotation=RoundRobin(),
):
    """returns working_thermoelectrics, defict per day, stored energy, circuits"""

    working_thermoelectrics = []

    working_thermoelectrics.append(np.ones(len(thermoelectrics)))

    deficit_per_day = []

    deficit_per_day.append(0)

    stored_energy_per_day = []

    stored_energy_per_day.append(0)

    event_date, event_thermoelectric = get_next_general_event(thermoelectrics, days, 0)

    for current_day in range(0, days):
        total_demand = 0

        if circuits is not None:
            total_demand = sum([c.get_demand(current_day) for c in circuits])

        if agent is not None:
            agent.Manage_Thermoelectrics(
                current_day, stored_energy, circuits, thermoelectrics, rotation
            )

        while event_thermoelectric != None and np.floor(event_date) == current_day:

            event_thermoelectric.pop_next_future_event()

            event_date, event_thermoelectric = get_next_general_event(
                thermoelectrics, days, current_day
            )

        thermoelectrics_state = [t.is_working() for t in thermoelectrics]

        working_thermoelectrics.append(thermoelectrics_state)

        if circuits is not None:

            if agent is not None:

                agent.Manage_Circuits(
                    current_day, stored_energy, circuits, thermoelectrics, rotation
                )

            total_offer = (
                sum([x.offer for x in thermoelectrics if x.is_working()])
                + stored_energy_per_day[-1]
            )

            deficit_today = max(total_demand - total_offer, 0)

            deficit_per_day.append(deficit_today)

            stored_energy = max(total_offer - total_demand, 0)

            stored_energy_per_day.append(stored_energy)

    return working_thermoelectrics, deficit_per_day, stored_energy_per_day, circuits

In [ ]:
# Running simulation without agent intervention and obtaining graphs

tmp = copy.deepcopy(thermoelectrics)
tmp_circuits = copy.deepcopy(circuits)


(
    working_thermoelectrics_without_strategy,
    deficit_per_day_without_strategy,
    stored_energy_without_strategy,
    circuits_without_strategy,
) = simulate(tmp, days, circuits=tmp_circuits, stored_energy=stored_energy)


working_thermoelectrics_without_strategy = [
    sum(x) for x in working_thermoelectrics_without_strategy
]


fig_without_heuristic = go.Figure()


fig_without_heuristic.add_trace(
    go.Scatter(
        x=list(range(0, days)),
        y=working_thermoelectrics_without_strategy,
        mode="lines",
    )
)


fig_without_heuristic.update_layout(
    title="Working thermoelectrics per day",
    xaxis_title="Days",
    yaxis_title="Working thermoelectrics",
)


fig_without_heuristic = go.Figure()

fig_without_heuristic.add_trace(
    go.Scatter(
        x=list(range(0, days)),
        y=stored_energy_without_strategy,
        mode="lines",
        name="Stored Energy",
    )
)
fig_without_heuristic.add_trace(
    go.Scatter(
        x=list(range(0, days)),
        y=deficit_per_day_without_strategy,
        mode="lines",
        name="Deficit",
    )
)

fig_without_heuristic.update_layout(
    title="Energy per day",
    xaxis_title="Days",
    yaxis_title="Energy",
)


fig_without_heuristic.show()

In [ ]:
def get_deficit_and_stored_energy(
    circuits: "list[Circuit]",
    thermoelectrics: "list[ThermoElectric]",
    stored_energy,
    current_day,
):
    total_demand = sum([x.get_demand(current_day) for x in circuits])
    total_offer = (
        sum([x.offer for x in thermoelectrics if x.is_working()]) + stored_energy
    )
    return (
        (total_demand - total_offer, 0)
        if total_demand > total_offer
        else (0, total_offer - total_demand)
    )

# Agent with thermoelectric maintenance strategy


In [ ]:
# Calculates the average working time of the thermoelectrics

def average_worktime(planification, days, amount):

    total_sum = 0
    total_intervals = 0

    for i in range(amount):
        sum = 0
        for j in range(days):
            if planification[j][i]:
                sum += 1
            elif sum != 0:
                total_sum += sum
                total_intervals += 1
                sum = 0
        if sum != 0:
            total_sum += sum
            total_intervals += 1
            sum = 0
    return total_sum, total_intervals

In [ ]:
# A function for running k simulations and obtaining stadistics

def k_simulation(days, thermoelectrics_amount, k, agent=None, circuits_amount=0):
    """returns working day average, thermoelectics average by day, deficit average by day, stored energy by day"""
    sum_time = 0
    intervals = 0

    average_active_thermoelectric = 0
    average_deficit = 0
    average_stored_energy = 0

    total_working_thermoelectrics_per_day = np.zeros(days + 1)
    total_deficit_per_day = np.zeros(days + 1)
    total_stored_energy_per_day = np.zeros(days + 1)

    for i in range(k):
        thermoelectrics = generate_thermoelectrics(days, thermoelectrics_amount)
        circuits = (
            None if circuits_amount <= 0 else generate_circuits(circuits_amount, days)
        )
        (
            thermoelectrics_state,
            deficit_per_day,
            stored_energy_per_day,
            circuits_result,
        ) = simulate(thermoelectrics, days, agent, circuits)
        (
            partial_sum_working_thermoelectrics,
            partial_intervals_working_thermoelectrics,
        ) = average_worktime(thermoelectrics_state, days, thermoelectrics_amount)

        sum_time += partial_sum_working_thermoelectrics
        intervals += partial_intervals_working_thermoelectrics

        number_of_working_thermoelectrics_current_simulation = [
            sum(x) for x in thermoelectrics_state
        ]

        total_working_thermoelectrics_per_day += np.array(
            number_of_working_thermoelectrics_current_simulation
        )
        total_deficit_per_day += np.array(deficit_per_day)
        total_stored_energy_per_day += np.array(stored_energy_per_day)

        average_active_thermoelectric += sum(
            number_of_working_thermoelectrics_current_simulation
        )
        average_deficit += sum(deficit_per_day)
        average_stored_energy += sum(stored_energy_per_day)

    working_average = sum_time / intervals if intervals != 0 else 0
    total_working_thermoelectrics_per_day /= k
    total_deficit_per_day /= k
    total_stored_energy_per_day /= k

    average_active_thermoelectric /= k * days
    average_deficit /= k * days
    average_stored_energy /= k * days

    return (
        working_average,
        total_working_thermoelectrics_per_day,
        total_deficit_per_day,
        total_stored_energy_per_day,
        average_active_thermoelectric,
        average_deficit,
        average_stored_energy,
    )

In [ ]:
tmp_thermoelectrics_maintenance = copy.deepcopy(thermoelectrics)
tmp_circuits_maintenance = copy.deepcopy(circuits)

average, _, _, _, _, _, _ = k_simulation(
    days, 20, 10, circuits_amount=len(tmp_circuits_maintenance)
)

average *= 2

# A function for giving maintenance to a thermoelectric if it has been 
# working for the average thermoelectric worktime an if there is no thermoelectric receiving
# maintenance at the time
def give_maintenance_heuristic(
    current_day,
    stored_energy,
    circuits: "list[Circuit]",
    thermoelectrics: "list[ThermoElectric]",
    rotation: "RoundRobin",
):
    if stored_energy <= 1e-8:
        return

    if sum([thermoelectric.is_on_maintenance() for thermoelectric in thermoelectrics]) >= 1:
        return

    for thermoelectric in thermoelectrics:

        if not thermoelectric.is_working():
            continue
        last_repair = thermoelectric.get_last_repair_day()
        if current_day - last_repair >= average:
            thermoelectric.repair_and_replanificate(
                current_day,
                days,
                LogNormal(rnd.uniform(1.5, 2), rnd.uniform(0.2, 0.4)),
            )
            maintainance_thermoelectric = thermoelectric
            return


def empty_func(arg0, arg1, arg2, arg3, rg4):
    pass

# Running simulation with agent and Maintenance heuristic
(
    working_thermoelectrics_maintenance_heuristic,
    deficit_per_day_maintenance_heuristic,
    stored_per_day_maintenance_heuristic,
    circuits_maintenance_heuristic,
) = simulate(
    tmp_thermoelectrics_maintenance,
    days,
    Agent(
        give_maintenance_heuristic,
        empty_func,
    ),
    tmp_circuits_maintenance,
    stored_energy,
)
working_thermoelectrics_maintenance_heuristic = [
    sum(x) for x in working_thermoelectrics_maintenance_heuristic
]

fig_maintenance_heuristic = go.Figure()
fig_maintenance_heuristic.add_trace(
    go.Scatter(
        x=list(range(0, days)),
        y=working_thermoelectrics_maintenance_heuristic,
        mode="lines",
    )
)
fig_maintenance_heuristic.update_layout(
    title="Maintenance heuristic",
    xaxis_title="Days",
    yaxis_title="Working thermoelectrics",
)

fig_maintenance_heuristic.show()

fig_energy_deficit_and_stored_maintenance_heuristic = go.Figure()

fig_energy_deficit_and_stored_maintenance_heuristic.add_trace(
    go.Scatter(
        x=list(range(0, days)),
        y=stored_per_day_maintenance_heuristic,
        name="Stored Energy",
    )
)

fig_energy_deficit_and_stored_maintenance_heuristic.add_trace(
    go.Scatter(
        x=list(range(0, days)),
        y=deficit_per_day_maintenance_heuristic,
        name="Deficit Energy",
    )
)


fig_energy_deficit_and_stored_maintenance_heuristic.update_layout(
    title="Energy per day",
    xaxis_title="Days",
    yaxis_title="Energy",
    legend=dict(x=0.7, y=0.95),
    showlegend=True,
)

# Agent with circuit strategies


In [ ]:
rotation = RoundRobin()

# A function for disconnecting circuits rotating through them and disconnecting 0.25 of the demand while there is still deficit
def disconnect_circuit_heuristic(
    current_day,
    stored_energy,
    circuits: "list[Circuit]",
    thermoelectrics: "list[ThermoElectric]",
    rotation: "RoundRobin",
):
    deficit, stored_energy = get_deficit_and_stored_energy(
        circuits, thermoelectrics, stored_energy, current_day
    )

    while deficit - EPSILON > 0:
        circuit = rotation.next(circuits)
        deficit -= circuit.disconnect(0.25, current_day)

In [ ]:
tmp_thermoelectrics = copy.deepcopy(thermoelectrics)
tmp_circuits = copy.deepcopy(circuits)

# Running simulation with both strategies
(
    working_thermoelectrics_both_heuristic,
    deficit_per_day_both_heuristic,
    stored_energy_per_day_both_heuristic,
    circuits_result,
) = simulate(
    tmp_thermoelectrics,
    days,
    Agent(
        give_maintenance_heuristic,
        disconnect_circuit_heuristic,
    ),
    tmp_circuits,
    stored_energy,
)
working_thermoelectrics_both_heuristic = [
    sum(x) for x in working_thermoelectrics_both_heuristic
]


fig_both_heuristic = go.Figure()
fig_both_heuristic.add_trace(
    go.Scatter(
        x=list(range(0, days)),
        y=working_thermoelectrics_both_heuristic,
        mode="lines",
    )
)
fig_both_heuristic.update_layout(
    title="Maintenance heuristic and disconnect circuit heuristic",
    xaxis_title="Days",
    yaxis_title="Working thermoelectrics",
)

fig_both_heuristic.show()


fig_energy_deficit_and_stored = go.Figure()


fig_energy_deficit_and_stored.add_trace(
    go.Scatter(
        x=list(range(0, days)),
        y=stored_energy_per_day_both_heuristic,
        mode="lines",
        name="Energy Stored per day",
    )
)

fig_energy_deficit_and_stored.add_trace(
    go.Scatter(
        x=list(range(0, days)),
        y=deficit_per_day_both_heuristic,
        mode="lines",
        name="Energy Deficit per day",
    )
)

fig_energy_deficit_and_stored.update_layout(
    title="Energy per day",
    xaxis_title="Days",
    yaxis_title="Energy",
    legend=dict(x=0.7, y=0.95),
    showlegend=True,
)


fig_energy_deficit_and_stored.show()

In [ ]:
# Plotting the total deficit and demand per circuit. Due to the implemented blackout strategy, these data are directly proportional.

total_deficit_by_circuit = [x.total_deficit for x in circuits_result]
demand_by_circuit = [sum(x.demand) / len(x.demand) for x in circuits_result]
x = list(range(len(total_deficit_by_circuit)))

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=x,
        y=total_deficit_by_circuit,
    )
)

fig.update_layout(
    title="Total Deficit per Circuit",
    xaxis_title="Circuit ID",
    yaxis_title="Deficit",
)

fig.show()

fig = go.Figure()

fig.add_trace(go.Bar(x=x, y=demand_by_circuit))
fig.update_layout(
    title="Total Demand per Circuit",
    xaxis_title="Circuit ID",
    yaxis_title="Demand",
)

fig.show()

In [ ]:
# Plotting some circuits' deficits

for c in circuits_result[:10]:
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=list(range(days)),
            y=c.deficit_history
        )
    )
    fig.update_layout(
        title=f'Deficit per day in Circuit {c.id}',
        xaxis_title='Days',
        yaxis_title="Deficit",
    )
    fig.show()

# Maintenance VS Non Maintenance


In [ ]:
list_of_days = list(range(0, days))
fig_comparison_working_thermoelectrics = go.Figure()
fig_comparison_working_thermoelectrics.add_trace(
    go.Scatter(
        x=list_of_days,
        y=working_thermoelectrics_maintenance_heuristic,
        mode="lines",
        name="Maintenance Strategy",
    )
)

fig_comparison_working_thermoelectrics.add_trace(
    go.Scatter(
        x=list_of_days,
        y=working_thermoelectrics_without_strategy,
        mode="lines",
        name="Without Maintenance Strategy",
    )
)

fig_comparison_energy = go.Figure()

fig_comparison_energy.add_trace(
    go.Scatter(
        x=list_of_days,
        y=stored_per_day_maintenance_heuristic,
        mode="lines",
        name="Maintenance Strategy",
    )
)

fig_comparison_energy.add_trace(
    go.Scatter(
        x=list_of_days,
        y=stored_energy_without_strategy,
        mode="lines",
        name="Without Maintenance Strategy",
    )
)

fig_comparison_deficit = go.Figure()

fig_comparison_deficit.add_trace(
    go.Scatter(
        x=list_of_days,
        y=deficit_per_day_maintenance_heuristic,
        mode="lines",
        name="Maintenance Strategy",
    )
)

fig_comparison_deficit.add_trace(
    go.Scatter(
        x=list_of_days,
        y=deficit_per_day_without_strategy,
        mode="lines",
        name="Without Maintenance Strategy",
    )
)

fig_comparison_working_thermoelectrics.update_layout(
    title="Working Thermoelectric per day",
    xaxis_title="Days",
    yaxis_title="Thermoeelctrics",
    legend=dict(x=0.7, y=0.95),
    showlegend=True,
)


fig_comparison_energy.update_layout(
    title="Stored Energy per day",
    xaxis_title="Days",
    yaxis_title="Energy",
    legend=dict(x=0.7, y=0.95),
    showlegend=True,
)


fig_comparison_deficit.update_layout(
    title="Deficit per day",
    xaxis_title="Days",
    yaxis_title="Energy",
    legend=dict(x=0.7, y=0.95),
    showlegend=True,
)

fig_comparison_working_thermoelectrics.show()
fig_comparison_energy.show()
fig_comparison_deficit.show()

## Repeat experiments


In [ ]:
# number of simulations
K_SIMULATIONS = 500

# non maintenance
(
    _,
    non_maintenance_average_of_working_per_day,
    non_maintenance_average_deficit_per_day,
    non_maintenance_average_stored_energy_per_day,
    non_maintenance_average_of_working,
    non_maintenance_average_deficit,
    non_maintenance_average_stored_energy,
) = k_simulation(days, thermoelectrics_amount, K_SIMULATIONS, None, circuits_amount)

# maintenance
(
    _,
    maintenance_average_of_working_per_day,
    maintenance_average_deficit_per_day,
    maintenance_average_stored_energy_per_day,
    maintenance_average_of_working,
    maintenance_average_deficit,
    maintenance_average_stored_energy,
) = k_simulation(
    days,
    thermoelectrics_amount,
    K_SIMULATIONS,
    Agent(give_maintenance_heuristic, empty_func),
    circuits_amount,
)

## print stadistics in md format

display(
    Markdown(
        f"""### Non Maintenance
Average in {K_SIMULATIONS} simulations:

`Working Thermoelectrics:` {non_maintenance_average_of_working}

`Deficit:` {non_maintenance_average_deficit}

`Stored Energy:` {non_maintenance_average_stored_energy}
"""
    )
)

display(
    Markdown(
        f"""### Maintenance
Average in {K_SIMULATIONS} simulations:

`Working Thermoelectrics:` {maintenance_average_of_working}

`Deficit:` {maintenance_average_deficit}

`Stored Energy:` {maintenance_average_stored_energy}
"""
    )
)


display(
    Markdown(
        f"""### Maintenance vs Non Maintenance
Average day in {K_SIMULATIONS} simulations"""
    )
)


list_of_days = list(range(days))

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=list_of_days, y=maintenance_average_of_working_per_day, name="Maintenance"
    )
)

fig.add_trace(
    go.Scatter(
        x=list_of_days,
        y=non_maintenance_average_of_working_per_day,
        name="Non Maintenance",
    )
)


fig.update_layout(
    title="Maintenance vs Non Maintenance: Day Average Working Thermoelectrics",
    xaxis_title="Days",
    yaxis_title="Working Thermoelectrics",
    legend=dict(x=0.7, y=0.95),
    showlegend=True,
)
fig.show()

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=list_of_days, y=maintenance_average_deficit_per_day, name="Maintenance"
    )
)
fig.add_trace(
    go.Scatter(
        x=list_of_days,
        y=non_maintenance_average_deficit_per_day,
        name="Non Maintenance",
    )
)


fig.update_layout(
    title="Maintenance vs Non Maintenance: Day Average Deficit",
    xaxis_title="Days",
    yaxis_title="Energy",
    legend=dict(x=0.7, y=0.95),
    showlegend=True,
)

fig.show()

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=list_of_days, y=maintenance_average_stored_energy_per_day, name="Maintenance"
    )
)

fig.add_trace(
    go.Scatter(
        x=list_of_days,
        y=non_maintenance_average_stored_energy_per_day,
        name="Non Maintenance",
    )
)


fig.update_layout(
    title="Maintenance vs Non Maintenance: Day Stored Energy",
    xaxis_title="Days",
    yaxis_title="Energy",
    legend=dict(x=0.7, y=0.95),
    showlegend=True,
)

fig.show()